In [1]:
# structure informations and init/goal position added
import json, os, re

problem_ids = []
init_poss = {}
goal_poss = {}
anss = {}
second_num = {(0,0,3):'1',(1,0,2):'2',(0,1,2):'3',(0,2,1):'4',(1,1,1):'5',(1,2,0):'6'}
R = 'r'
G = 'g'
B = 'b'
ID2color = {11:{(2,0):R,(2,1):G, (2,2):B},12:{(2,0):R,(2,1):G,(0,0):B}, 13:{(2,0):R,(2,1):G,(1,0):B}, 14:{(2,0):R,(1,0):B,(1,1):G}, 15:{(2,0):R,(1,0):B,(0,0):G}, 16:{(1,0):B,(1,1):R,(0,0):G},
           21:{(2,0):R,(2,1):B, (2,2):G},22:{(2,0):R,(2,1):B,(0,0):G}, 23:{(2,0):R,(2,1):B,(1,0):G}, 24:{(2,0):R,(1,0):G,(1,1):B}, 25:{(2,0):R,(1,0):G,(0,0):B}, 26:{(1,0):G,(1,1):R,(0,0):B},
            31:{(2,0):B,(2,1):R, (2,2):G},32:{(2,0):B,(2,1):R,(0,0):G}, 33:{(2,0):B,(2,1):R,(1,0):G}, 34:{(2,0):B,(1,0):G,(1,1):R}, 35:{(2,0):B,(1,0):G,(0,0):R}, 36:{(1,0):G,(1,1):B,(0,0):R},
            41:{(2,0):B,(2,1):G, (2,2):R},42:{(2,0):B,(2,1):G,(0,0):R}, 43:{(2,0):B,(2,1):G,(1,0):R}, 44:{(2,0):B,(1,0):R,(1,1):G}, 45:{(2,0):B,(1,0):R,(0,0):G}, 46:{(1,0):R,(1,1):B,(0,0):G},
            51:{(2,0):G,(2,1):B, (2,2):R},52:{(2,0):G,(2,1):B,(0,0):R}, 53:{(2,0):G,(2,1):B,(1,0):R}, 54:{(2,0):G,(1,0):R,(1,1):B}, 55:{(2,0):G,(1,0):R,(0,0):B}, 56:{(1,0):R,(1,1):G,(0,0):B},
            61:{(2,0):G,(2,1):R, (2,2):B},62:{(2,0):G,(2,1):R,(0,0):B}, 63:{(2,0):G,(2,1):R,(1,0):B}, 64:{(2,0):G,(1,0):B,(1,1):R}, 65:{(2,0):G,(1,0):B,(0,0):R}, 66:{(1,0):B,(1,1):G,(0,0):R}}

data_string = open("berg/test.txt").read().strip().split("\n\n")
data_items = [line.strip().split("\n") for line in data_string]

count = 0
for folder in os.listdir("TOLdataset"):
    problem_file = "TOLdataset/" + folder + "/problem.pddl"

    problem = open(problem_file).read()
    init_string = problem[problem.find("(:init") + 6 :problem.find("(:goal")].strip() 
    infos = init_string.split("\n")[:-1]
    pos = [[None] * 3 for i in range(3)]
    height_tuple = [0]*3
    for info in infos:
        if info.strip()[1] == 'a':
            tmp = info.strip()[1:-2].split('-')
            box = tmp[1]
            loc = tmp[2]

            x,y = loc.split('_')[1:]

            pos[int(x)][int(y)] = box.split('_')[1]

    if pos[0][0]:
        height_tuple[0] = 1
    if pos[1][1]:
        height_tuple[1] = 2
    elif pos[1][0]:
        height_tuple[1] = 1
        
    init_pos = [[0,0,0],[0,0],[0]]
    for i in range(len(init_pos)):
        for j in range(0,3-i):
            if pos[2-i][j]:
                init_pos[i][j] = int(pos[2-i][j]) + 1
                
    print(init_pos)
        
        
    height_tuple[2] = 3 - height_tuple[0] - height_tuple[1]
        
    num_2 = second_num[(height_tuple[0], height_tuple[1],height_tuple[2])]
    start_num = int('2' + num_2)
    color = ID2color[start_num]
    num2color = {}
    for (x,y) in color:
        num2color[pos[x][y]] = color[(x,y)]
    


    goal_string = problem[problem.find("(:goal"):-1].strip()
    infos = re.findall("at-b_\d-loc_\d_\d",goal_string)
    pos = [[None] * 3 for i in range(3)]
    for info in infos:
        if info.strip()[0] == 'a':
            tmp = info.strip().split('-')
            box = tmp[1]
            loc = tmp[2]

            x,y = loc.split('_')[1:]

            pos[int(x)][int(y)] = box.split('_')[1]
    
    goal_pos = [[0,0,0],[0,0],[0]]
    for i in range(len(goal_pos)):
        for j in range(0,3-i):
            if pos[2-i][j]:
                goal_pos[i][j] = int(pos[2-i][j]) + 1
                
    print(goal_pos)
            
    for num in ID2color:
        tmp = ID2color[num]
        for (x,y) in tmp:
            if (not pos[x][y]) or (not num2color[pos[x][y]] == tmp[(x,y)]):
                break
        else:
            end_num = num
            break
            
    info_dict = {}

    
    
    for line in data_items:
        if line[0] == str(start_num) and line[1] == str(end_num):
            print(folder)
            print(line)

            info_dict['num_sols'] = int(line[3])
            info_dict['alter_move'] = float(line[4])
            info_dict['start_hierarchy'] = int(line[5]) - 1
            info_dict['goal_hierarchy'] = int(line[6]) - 1
            
            info_dict['berg_score'] = float(line[-1])
            print(count)
            break
    
    info_dict['init_pos'] = init_pos
    info_dict['goal_pos'] = goal_pos
            
    with open("TOLdataset/" + folder + "/info.json", 'w') as f:
        json.dump(info_dict, f)
    


[[3, 0, 0], [1, 2], [0]]
[[1, 0, 0], [2, 3], [0]]
TOL_87
['24', '64', '5', '1', '3.08', '2', '2', '0.899', '0.545', '0.400', '2.792']
0
[[3, 0, 0], [2, 0], [1]]
[[3, 2, 0], [1, 0], [0]]
[[2, 3, 0], [1, 0], [0]]
[[2, 0, 0], [3, 1], [0]]
[[1, 2, 3], [0, 0], [0]]
[[2, 3, 0], [0, 0], [1]]
TOL_18
['21', '42', '5', '1', '3.00', '3', '2', '-0.515', '-0.170', '-1.044', '-0.533']
0
[[0, 0, 0], [2, 3], [1]]
[[1, 2, 0], [3, 0], [0]]
TOL_190
['26', '43', '4', '1', '3.20', '2', '2', '-0.598', '-0.297', '-0.738', '-1.158']
0
[[1, 2, 3], [0, 0], [0]]
[[0, 0, 0], [2, 1], [3]]
[[2, 3, 0], [0, 0], [1]]
[[0, 0, 0], [3, 2], [1]]
[[3, 0, 0], [2, 0], [1]]
[[1, 2, 3], [0, 0], [0]]
TOL_144
['25', '41', '5', '2', '3.00', '1', '3', '-0.667', '-0.680', '-0.374', '-0.588']
0
[[3, 0, 0], [2, 0], [1]]
[[2, 1, 0], [3, 0], [0]]
TOL_155
['25', '53', '7', '1', '3.38', '1', '2', '0.760', '0.888', '0.015', '1.007']
0
[[2, 3, 0], [0, 0], [1]]
[[3, 0, 0], [1, 2], [0]]
TOL_120
['22', '34', '4', '1', '3.20', '2', '2', '-1.52

In [6]:
# 4caps info added
import json
for folder in os.listdir("TOLdataset"):      
    info_file = "TOLdataset/" + folder + "/model_all_info.txt"
    infos = open(info_file).read().split("\n")
    last = None
    for i in range(1,len(infos)):
        tmp = infos[-1 - i]
        if "END-PUZZLE" in tmp:
            
            break
        else:
            if "(PUZZLE :ID" in tmp:
                last = tmp
    
    action_str = None
    if last:
        action_str = last[last.find("((")+1: -2]
    
    print(action_str)
    print(folder)
    
    nums = [0, 0, 0]
    if action_str:
        for i in action_str.split(') ('):
            infos = i.split(' ')
            for index, num in enumerate(infos):
                if num.isdigit():
                    nums[index] += 1
        print(nums)

        with open("TOLdataset/" + folder + "/info.json") as f:
            info_dict = json.load(f)

        info_dict["action_count(4caps)"] = {str(nums[0]) + str(nums[1]) + str(nums[2]):1}

    else:
        with open("TOLdataset/" + folder + "/info.json") as f:
            info_dict = json.load(f)

        info_dict["action_count(4caps)"] = {}

    with open("TOLdataset/" + folder + "/model_time.txt") as f:
        info_dict["time(4caps)"] = int(f.readline())
    
    with open("TOLdataset/" + folder + "/info.json", 'w') as f:
        json.dump(info_dict, f)

(NIL NIL NIL) (1 0 NIL) (2 NIL NIL)
TOL_87
[2, 1, 0]
(NIL NIL 0) (1 NIL NIL) (2 NIL NIL)
TOL_151
[2, 0, 1]
(NIL NIL 0) (2 NIL NIL) (1 NIL NIL)
TOL_52
[2, 0, 1]
(NIL NIL NIL) (1 2 NIL) (0 NIL NIL)
TOL_18
[2, 1, 0]
(NIL 2 NIL) (NIL 1 NIL) (0 NIL NIL)
TOL_190
[1, 2, 0]
(NIL NIL 2) (1 NIL NIL) (0 NIL NIL)
TOL_35
[2, 0, 1]
(NIL NIL 0) (NIL 2 NIL) (1 NIL NIL)
TOL_139
[1, 1, 1]
(NIL 2 0) (NIL 1 NIL) (NIL NIL NIL)
TOL_144
[0, 2, 1]
(NIL NIL 0) (1 NIL NIL) (2 NIL NIL)
TOL_155
[2, 0, 1]
(NIL NIL NIL) (2 0 NIL) (1 NIL NIL)
TOL_120
[2, 1, 0]
(NIL 2 NIL) (NIL 0 NIL) (1 NIL NIL)
TOL_45
[1, 2, 0]
(NIL 2 NIL) (NIL 0 NIL) (1 NIL NIL)
TOL_57
[1, 2, 0]
(NIL 2 0) (NIL 1 NIL) (NIL NIL NIL)
TOL_161
[0, 2, 1]
(NIL NIL 0) (2 NIL NIL) (1 NIL NIL)
TOL_65
[2, 0, 1]
(NIL 2 0) (NIL 1 NIL) (NIL NIL NIL)
TOL_152
[0, 2, 1]
(NIL NIL NIL) (1 2 NIL) (0 NIL NIL)
TOL_11
[2, 1, 0]
(NIL NIL 0) (NIL 2 NIL) (1 NIL NIL)
TOL_124
[1, 1, 1]
(NIL 2 NIL) (NIL 0 NIL) (1 NIL NIL)
TOL_66
[1, 2, 0]
(NIL NIL NIL) (2 0 NIL) (1 NIL NIL)
T